In [1]:
Doc1 = ["""Wasting natural resources is a very serious problem, since we all know that natural resources are limited but still we dont feel it. We use it more than we need, Government are also encouraging people to save the natural resoucres""" ]
Doc2 = ["""Machine learning is now days a very popular field of study, a continuous reasearch is going on this , Machine learning is all about maths. Analysing the patters solve the task."""]
Doc3 = ["""Nowdays books are loosing its charm since the phones and smart gadgets has taken over the old time, Books are now printed in Digital ways, This saves papers and ultimately saves thousands of trees"""]
Doc4 = ["""The man behind the wicket is MS DHONI , his fast hand behind wicket are a big advantage for india, but pant has now carrying the legacy of DHONI but he is not that fast"""]

In [2]:
#example with a very small dataset made by 4 docs
data = Doc1 + Doc2 + Doc3 + Doc4
print(data)

['Wasting natural resources is a very serious problem, since we all know that natural resources are limited but still we dont feel it. We use it more than we need, Government are also encouraging people to save the natural resoucres', 'Machine learning is now days a very popular field of study, a continuous reasearch is going on this , Machine learning is all about maths. Analysing the patters solve the task.', 'Nowdays books are loosing its charm since the phones and smart gadgets has taken over the old time, Books are now printed in Digital ways, This saves papers and ultimately saves thousands of trees', 'The man behind the wicket is MS DHONI , his fast hand behind wicket are a big advantage for india, but pant has now carrying the legacy of DHONI but he is not that fast']


In [3]:
import numpy as np
import nltk
import itertools
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy
from scipy import spatial
import re
tokenizer = nltk.ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

In [4]:
def remove_stopwords(text, is_lower_case=False):
    pattern = r'[^a-zA-z0-9s]'
    text = re.sub(pattern," ",''.join(text))
    tokens = tokenizer.tokenize(text)
    tokens = [tok.strip() for tok in tokens]
    if is_lower_case:
        cleaned_token = [tok for tok in tokens if tok not in stopword_list]
    else:
        cleaned_tokens = [tok for tok in tokens if tok.lower() not in stopword_list]
    filtered_text = ' '.join(cleaned_tokens)
    return filtered_text

![word embeddings](w2vec.png)

In [5]:
glove_vectors = dict()
file = open('inputs/glove6b/glove.6B.300d.txt', encoding = 'utf-8')
for line in file:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1:])
    glove_vectors[word] = vectors
file.close()

In [6]:
#example
glove_vectors['cat']

array(['-0.29353', '0.33247', '-0.047372', '-0.12247', '0.071956',
       '-0.23408', '-0.06238', '-0.0037192', '-0.39462', '-0.69411',
       '0.36731', '-0.12141', '-0.044485', '-0.15268', '0.34864',
       '0.22926', '0.54361', '0.25215', '0.097972', '-0.087305',
       '0.87058', '-0.12211', '-0.079825', '0.28712', '-0.68563',
       '-0.27265', '0.22056', '-0.75752', '0.56293', '0.091377',
       '-0.71004', '-0.3142', '-0.56826', '-0.26684', '-0.60102',
       '0.26959', '-0.17992', '0.10701', '-0.57858', '0.38161',
       '-0.67127', '0.10927', '0.079426', '0.022372', '-0.081147',
       '0.011182', '0.67089', '-0.19094', '-0.33676', '-0.48471',
       '-0.35406', '-0.15209', '0.44503', '0.46385', '0.38409',
       '0.045081', '-0.59079', '0.21763', '0.38576', '-0.44567',
       '0.009332', '0.442', '0.097062', '0.38005', '-0.11881', '-0.42718',
       '-0.31005', '-0.025058', '0.12689', '-0.13468', '0.11976',
       '0.76253', '0.2524', '-0.26934', '0.068629', '-0.10071',
     

In [7]:
vec_dimension = 300
def get_embedding(x):
    arr  = np.zeros(vec_dimension)
    text = str(x).split()
    for t in text:
        try:
            vec = glove_vectors.get(t).astype(float)
            arr = arr + vec
        except:
            pass
    arr = arr.reshape(1,-1)[0]
    return(arr/len(text))

In [8]:
out_dict = {}
for sen in data:
    average_vector = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(remove_stopwords(sen))]), axis=0))
    dict = { sen : (average_vector) }
    out_dict.update(dict)

In [14]:
out_dict

{'Wasting natural resources is a very serious problem, since we all know that natural resources are limited but still we dont feel it. We use it more than we need, Government are also encouraging people to save the natural resoucres': array([ 9.95549545e-03,  1.17805139e-01, -3.61807727e-02,  1.01260455e-02,
        -1.14285909e-03,  2.58203955e-02,  4.38876364e-02,  1.54597273e-02,
         7.47564545e-02, -1.53129223e+00,  1.14993036e-01, -5.34365909e-02,
        -2.50636027e-01,  6.23110455e-02,  3.25380000e-03, -1.23839591e-01,
        -1.79103773e-01,  1.07420041e-01, -5.20865455e-02, -2.55601364e-02,
        -4.47790909e-02,  2.11833455e-01,  1.36536736e-01,  4.57030909e-02,
        -1.82952773e-01,  7.16413182e-02,  5.90921364e-02,  5.43328636e-02,
        -9.25686818e-02,  1.65448127e-01,  3.07184409e-01,  3.33741350e-01,
        -1.94431773e-01, -2.46050818e-01, -5.17668155e-01, -6.81972864e-02,
         8.37739773e-02, -4.57758864e-02,  1.20631364e-02, -4.00177273e-03,
      

In [15]:
def get_sim(query_embedding, average_vector_doc):
    sim = [(1 - scipy.spatial.distance.cosine(query_embedding, 
    average_vector_doc))]
    return sim

In [16]:
def Ranked_documents(query):
    query_word_vectors = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(query.lower())],dtype=float), axis=0))
    rank = []
    for k,v in out_dict.items():
        rank.append((k, get_sim(query_word_vectors, v)))
        rank = sorted(rank,key=lambda t: t[1], reverse=True)
    print('Ranked Documents :')
    return rank[0]

In [17]:
Ranked_documents('cat and ball')

Ranked Documents :


('The man behind the wicket is MS DHONI , his fast hand behind wicket are a big advantage for india, but pant has now carrying the legacy of DHONI but he is not that fast',
 [0.6044627331596946])

In [18]:
Ranked_documents('computer science')

Ranked Documents :


('Machine learning is now days a very popular field of study, a continuous reasearch is going on this , Machine learning is all about maths. Analysing the patters solve the task.',
 [0.5818178832764596])

In [22]:
Ranked_documents('science')

Ranked Documents :


('Machine learning is now days a very popular field of study, a continuous reasearch is going on this , Machine learning is all about maths. Analysing the patters solve the task.',
 [0.5177806955513218])